# Определение стоимости автомобиля

Для сервиса по продаже автомобилей необходимо создать модель предсказания рыночной цены, чтобы клиенты сервиса могли самостоятельно определять стоимость своего автомобиля в приложении.

Заказчику важны:

- качество предсказания;
- скорость предсказания;
- время обучения.

Признаки:
- DateCrawled — дата скачивания анкеты из базы;
- VehicleType — тип автомобильного кузова;
- RegistrationYear — год регистрации автомобиля;
- Gearbox — тип коробки передач;
- Power — мощность (л. с.);
- Model — модель автомобиля;
- Kilometer — пробег (км);
- RegistrationMonth — месяц регистрации автомобиля;
- FuelType — тип топлива;
- Brand — марка автомобиля;
- NotRepaired — была машина в ремонте или нет;
- DateCreated — дата создания анкеты;
- NumberOfPictures — количество фотографий автомобиля;
- PostalCode — почтовый индекс владельца анкеты (пользователя);
- LastSeen — дата последней активности пользователя.

Целевой признак:
- Price — цена (евро).

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-Случайного-Леса-Регрессии" data-toc-modified-id="Модель-Случайного-Леса-Регрессии-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Модель Случайного Леса Регрессии</a></span></li><li><span><a href="#Модели-градиентного-бустинга" data-toc-modified-id="Модели-градиентного-бустинга-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модели градиентного бустинга</a></span><ul class="toc-item"><li><span><a href="#LightGBM" data-toc-modified-id="LightGBM-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>LightGBM</a></span></li><li><span><a href="#CatBoost" data-toc-modified-id="CatBoost-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>CatBoost</a></span></li><li><span><a href="#Используем-функционал-работы-с-категориальными-признаками--у-моделей-Catboost-и-lightGBM" data-toc-modified-id="Используем-функционал-работы-с-категориальными-признаками--у-моделей-Catboost-и-lightGBM-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Используем функционал работы с категориальными признаками  у моделей Catboost и lightGBM</a></span><ul class="toc-item"><li><span><a href="#Модель-LightGBM-с-категориальными-признаками" data-toc-modified-id="Модель-LightGBM-с-категориальными-признаками-2.2.3.1"><span class="toc-item-num">2.2.3.1&nbsp;&nbsp;</span>Модель LightGBM с категориальными признаками</a></span></li><li><span><a href="#Модель-CatBoost-с-категориальными-признаками" data-toc-modified-id="Модель-CatBoost-с-категориальными-признаками-2.2.3.2"><span class="toc-item-num">2.2.3.2&nbsp;&nbsp;</span>Модель CatBoost с категориальными признаками</a></span></li></ul></li></ul></li></ul></li><li><span><a href="#Анализ-моделей" data-toc-modified-id="Анализ-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Анализ моделей</a></span><ul class="toc-item"><li><span><a href="#Модель-случайного-леса" data-toc-modified-id="Модель-случайного-леса-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Модель случайного леса</a></span></li><li><span><a href="#Модель-LightGBM" data-toc-modified-id="Модель-LightGBM-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Модель LightGBM</a></span></li><li><span><a href="#Модель-CatBoost" data-toc-modified-id="Модель-CatBoost-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Модель CatBoost</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Вывод</a></span></li></ul></div>

## Подготовка данных

In [1]:
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from tqdm.notebook import tqdm
import time

#import xgboost as xgb
import lightgbm as lgb
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor

In [2]:
path = 'C:/users/v_kly'

In [3]:
df = pd.read_csv(path + '/datasets/autos.csv')

In [4]:
df.head()

,DateCrawled,Price,VehicleType,RegistrationYear,Gearbox,Power,Model,Kilometer,RegistrationMonth,FuelType,Brand,NotRepaired,DateCreated,NumberOfPictures,PostalCode,LastSeen
0,2016-03-24 11:52:17,480,NaN,1993,manual,0,golf,150000,0,petrol,volkswagen,NaN,2016-03-24 00:00:00,0,70435,2016-04-07 03:16:57
1,2016-03-24 10:58:45,18300,coupe,2011,manual,190,NaN,125000,5,gasoline,audi,yes,2016-03-24 00:00:00,0,66954,2016-04-07 01:46:50
2,2016-03-14 12:52:21,9800,suv,2004,auto,163,grand,125000,8,gasoline,jeep,NaN,2016-03-14 00:00:00,0,90480,2016-04-05 12:47:46
3,2016-03-17 16:54:04,1500,small,2001,manual,75,golf,150000,6,petrol,volkswagen,no,2016-03-17 00:00:00,0,91074,2016-03-17 17:40:17
4,2016-03-31 17:25:20,3600,small,2008,manual,69,fabia,90000,7,gasoline,skoda,no,2016-03-31 00:00:00,0,60437,2016-04-06 10:17:21


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 354369 entries, 0 to 354368
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   DateCrawled        354369 non-null  object
 1   Price              354369 non-null  int64 
 2   VehicleType        316879 non-null  object
 3   RegistrationYear   354369 non-null  int64 
 4   Gearbox            334536 non-null  object
 5   Power              354369 non-null  int64 
 6   Model              334664 non-null  object
 7   Kilometer          354369 non-null  int64 
 8   RegistrationMonth  354369 non-null  int64 
 9   FuelType           321474 non-null  object
 10  Brand              354369 non-null  object
 11  NotRepaired        283215 non-null  object
 12  DateCreated        354369 non-null  object
 13  NumberOfPictures   354369 non-null  int64 
 14  PostalCode         354369 non-null  int64 
 15  LastSeen           354369 non-null  object
dtypes: int64(7), object(

По целевому значению пропусков нет, но необходимо проверить есть ли нулевые значения, которые будут вносить шумы в обучение модели.

In [6]:
len(df['Price'].loc[df['Price'] == 0]) / len(df)

0.030397692800442477

Таких строк оказалось 3%, можем смело от них избавиться.

In [7]:
df = df.loc[df['Price'] != 0]

In [8]:
fetures_col = ['VehicleType', 'RegistrationYear', 'Gearbox',
       'Power', 'Model', 'Kilometer', 'RegistrationMonth', 'FuelType', 'Brand',
       'NotRepaired']

**Обработаем пропуски**

In [9]:
df.shape

(343597, 16)

In [10]:
df[fetures_col].isnull().mean()

VehicleType          0.098226
RegistrationYear     0.000000
Gearbox              0.050510
Power                0.000000
Model                0.050993
Kilometer            0.000000
RegistrationMonth    0.000000
FuelType             0.085737
Brand                0.000000
NotRepaired          0.191696
dtype: float64

Видно, что в признаки `NotRepaired` (была машина в ремонте или нет) почти 20% пропущенных значений. В данном случае, такое большое количество пропусков может быть связано с тем, что при такой постановке вопроса, для машин которые не были в ремонте данная графа просто не заполнялась. Таким образом логично будет заполнить пропуски значением `no`.

In [11]:
df.NotRepaired.fillna('no', inplace=True)

Остальные пропуски в четырех признаках имеют не большое процентное отношение к выборке, избавимся от них, так как общий размер выборки достаточно большой, а без пропусков работа моделей будет чище.

In [12]:
df = df.dropna(axis=0)

In [13]:
df[fetures_col].shape

(279085, 10)

In [14]:
1 - 279085/343597

0.18775484070000614

После удаления всех строк с пропусками, мы потеряли чуть менее 19% выборки, оставшее количество объектов 279 085 строк.

Проверим итоговый результат.

In [15]:
df[fetures_col].isnull().mean()

VehicleType          0.0
RegistrationYear     0.0
Gearbox              0.0
Power                0.0
Model                0.0
Kilometer            0.0
RegistrationMonth    0.0
FuelType             0.0
Brand                0.0
NotRepaired          0.0
dtype: float64

Пропуски отсутствуют.

**Поделим данные на тренировочную, валидационную и тестовую выборки**

In [16]:
features = df[fetures_col]
target = df['Price']

Предварительно произведем кодирование категориальных признаков.

In [17]:
features = pd.get_dummies(features, drop_first=True) 

In [18]:
features_train, features_valid_test, target_train, target_valid_test = train_test_split(
    features, target, test_size=0.4, random_state=12345)

features_valid, features_test, target_valid, target_test = train_test_split(
    features_valid_test, target_valid_test, test_size=0.5, random_state=12345)

In [19]:
print(features_train.shape, target_train.shape)
print(features_valid.shape, target_valid.shape)
print(features_test.shape, target_test.shape)

(167451, 306) (167451,)
(55817, 306) (55817,)
(55817, 306) (55817,)


## Обучение моделей

In [20]:
random_state = 42

Посчитаем среднюю цену в выборке.

In [21]:
df.Price.mean()

4934.819721590196

### Модель Случайного Леса Регрессии

Обучим модель RandomForestRegressor с параметрами по умолчанию и посчитаем контрольную метрику RMSE на валидационной выборке.

In [22]:
%%time
model_rfr_default = RandomForestRegressor(random_state=random_state)
model_rfr_default.fit(features_train, target_train)

Wall time: 5min 36s


RandomForestRegressor(random_state=42)

In [23]:
predictions = model_rfr_default.predict(features_valid)

In [24]:
rmse = mean_squared_error(target_valid, predictions)**0.5

In [25]:
print('RMSE модели Случайного Леса:', rmse)

RMSE модели Случайного Леса: 1573.9717486406037


**Подберем гиперпараметры и посчитаем изменение метрики.**

In [26]:
# формирования списка моделей с разными гиперпараметрами
def get_models_rfr():
    models = []
    for n in range(10, 100, 10):
        for i in range(2, 20, 3):
            models.append(RandomForestRegressor(n_estimators=n, max_depth = i))
    return models

In [27]:
# wall time: 1h 7min 53s
'''
%%time
models = get_models_rfr()
best_model_rfr = []
best_rmse = 10000
best_total_time = 0
for i in tqdm(models):
    startTime = time.time() 
    i.fit(features_train, target_train)
    endTime = time.time()
    totalTime = endTime - startTime 
    pred = i.predict(features_valid)
    rmse = mean_squared_error(target_valid, pred)**0.5
    if rmse < best_rmse:
        best_rmse = rmse
        best_model_rfr = i
        best_total_time = totalTime
        
print('Время обучения лучшей модели:', round(best_total_time), 'сек.')
print('RMSE модели', best_rmse)
print('Лучшая модель:', best_model_rfr)
'''

"\n%%time\nmodels = get_models_rfr()\nbest_model_rfr = []\nbest_rmse = 10000\nbest_total_time = 0\nfor i in tqdm(models):\n    startTime = time.time() \n    i.fit(features_train, target_train)\n    endTime = time.time()\n    totalTime = endTime - startTime \n    pred = i.predict(features_valid)\n    rmse = mean_squared_error(target_valid, pred)**0.5\n    if rmse < best_rmse:\n        best_rmse = rmse\n        best_model_rfr = i\n        best_total_time = totalTime\n        \nprint('Время обучения лучшей модели:', round(best_total_time), 'сек.')\nprint('RMSE модели', best_rmse)\nprint('Лучшая модель:', best_model_rfr)\n"

Итоги работы ячейки выше:

Время обучения лучшей модели: 166 сек.

RMSE модели 1641.8401792795123

Лучшая модель: RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=17,

                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=80,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)
                      
CPU times: user 1h 6min 44s, sys: 11.7 s, total: 1h 6min 55s

Wall time: 1h 7min 53s

In [28]:
# Наилучшую модель Случайного леса сохраним в ручную, чтобы избежать перезапуска коды выше.
best_model_rfr = RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=17,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=80,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)


**Вывод:** Модель Случайного Леса Регрессии с подобраными гиперпараметрами обучилась за ``2 минуты 46 секунд``. Метрика RMSE равна `1641.84`

### Модели градиентного бустинга

####  LightGBM 

Обучим модель `LightGBM` с гиперпараметрами по умолчанию.

In [29]:
model_lgbm_default = LGBMRegressor()

In [30]:
%%time
model_lgbm_default.fit(features_train, target_train)

Wall time: 1.14 s


LGBMRegressor()

In [31]:
%%time
predictions = model_lgbm_default.predict(features_valid)

Wall time: 254 ms


In [32]:
print('RMSE модели LightGBM:', mean_squared_error(target_valid, predictions)**0.5)

RMSE модели LightGBM: 1646.2189752279303


Время обучения модели 1 минута 18 секунд. RMSE модели: 1760.8

**Изменим количество используемых деревьев в модели.**

Обучим четыре модели с 10, 50, 100 и 500 деревьями соответственно, и посмотрим как меняется метрика качества.

In [33]:
# формирования списка моделей с разными гиперпараметрами
def get_models():
    models = []
    trees = [10, 50, 100, 500]
    for n in trees:
        models.append(LGBMRegressor(n_estimators=n))
    return models

In [34]:
%%time
models = get_models()
for i in tqdm(models):
    startTime = time.time() 
    i.fit(features_train, target_train)
    endTime = time.time()
    totalTime = endTime - startTime 
    pred = i.predict(features_valid)
    rmse = mean_squared_error(target_valid, pred)**0.5
    print('Время обучения модели:', round(totalTime), 'сек.')
    print('RMSE модели', rmse)
    print()

Время обучения модели: 1 сек.
RMSE модели 2621.3507193507276

Время обучения модели: 1 сек.
RMSE модели 1737.0893824320453

Время обучения модели: 1 сек.
RMSE модели 1646.2189752279303

Время обучения модели: 2 сек.
RMSE модели 1532.3475695996954


Wall time: 5.8 s


Выше видим соотношение времени обучения модели и получившейся метрики качества RMSE.

#### CatBoost

Обучим модель `CatBoost` с параметрами по умолчанию.

In [35]:
cat_default = CatBoostRegressor()

In [36]:
%%time
cat_default.fit(features_train, target_train, verbose=100)

Learning rate set to 0.097108
0:	learn: 4324.6365107	total: 157ms	remaining: 2m 36s
100:	learn: 1777.7969933	total: 1.6s	remaining: 14.2s
200:	learn: 1677.7913581	total: 3s	remaining: 11.9s
300:	learn: 1628.6957680	total: 4.38s	remaining: 10.2s
400:	learn: 1593.6725136	total: 5.76s	remaining: 8.61s
500:	learn: 1568.9456971	total: 7.15s	remaining: 7.12s
600:	learn: 1547.2997339	total: 8.52s	remaining: 5.66s
700:	learn: 1529.2309581	total: 9.92s	remaining: 4.23s
800:	learn: 1513.3887016	total: 11.3s	remaining: 2.81s
900:	learn: 1500.5063880	total: 12.7s	remaining: 1.39s
999:	learn: 1488.8924240	total: 14s	remaining: 0us
Wall time: 14.7 s


In [37]:
predictions = cat_default.predict(features_valid)

In [38]:
rmse = mean_squared_error(target_valid, predictions)**0.5

In [39]:
 print('RMSE модели', rmse)

RMSE модели 1549.9199949174842


Время обучения модели 2 минуты. RMSE модели: 1759.95

**Переберем несколько параметров для `CatBoost`**

In [40]:
'''
%%time
cat = CatBoostRegressor()
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6]
        }

grid_search_result = cat.grid_search(grid, X=features_train, y=target_train)
'''

"\n%%time\ncat = CatBoostRegressor()\ngrid = {'learning_rate': [0.03, 0.1],\n        'depth': [4, 6]\n        }\n\ngrid_search_result = cat.grid_search(grid, X=features_train, y=target_train)\n"

Подбор параметров с обучением длился более 13 минут. 

Наилучшие подобранные параметры: `params': {'depth': 6, 'learning_rate': 0.1}`

Обучим модель с подобранными гиперпараметрами и посчитаем RMSE

In [41]:
cat_01_6 = CatBoostRegressor(learning_rate=0.1 , depth=6 )

In [42]:
%%time
cat_01_6.fit(features_train, target_train, verbose=100)

0:	learn: 4315.5882965	total: 16.2ms	remaining: 16.2s
100:	learn: 1775.3725093	total: 1.45s	remaining: 12.9s
200:	learn: 1675.6836506	total: 2.84s	remaining: 11.3s
300:	learn: 1624.5980018	total: 4.22s	remaining: 9.8s
400:	learn: 1590.7623421	total: 5.58s	remaining: 8.34s
500:	learn: 1563.8366346	total: 6.96s	remaining: 6.93s
600:	learn: 1542.7349389	total: 8.35s	remaining: 5.54s
700:	learn: 1525.6108606	total: 9.71s	remaining: 4.14s
800:	learn: 1510.9165023	total: 11.1s	remaining: 2.75s
900:	learn: 1497.8231532	total: 12.5s	remaining: 1.37s
999:	learn: 1485.7938096	total: 13.8s	remaining: 0us
Wall time: 14.7 s


In [43]:
predictions = cat_01_6.predict(features_valid)

In [44]:
rmse = mean_squared_error(target_valid, predictions)**0.5

In [45]:
print('RMSE модели', rmse)

RMSE модели 1549.398561305265


Модель обучалась 1 минуту 56 секунд. 

RMSE модели: 1670.05

**Вывод:** Были обучены несколько моделей разного типа с подбором гиперпараметров. Удалось получить результаты по скорости обучения и значению метрики RMSE. На следующем этапе проведем окончательные испытания на тестовой выборке и проведем анализ работы моделей с учетом целевой метрики.

#### Используем функционал работы с категориальными признаками  у моделей Catboost и lightGBM

Возьмем исходные данные после предобработки, но до кодирования.

In [46]:
fetures_not_encoded = df[fetures_col]

Выделяем категориальный признаки.

In [47]:
category = ['VehicleType', 'Gearbox', 'Model', 'FuelType', 'Brand', 'NotRepaired']

Меняем тип данных на ``category``.

In [48]:
fetures_not_encoded[category] = fetures_not_encoded[category].astype('category').copy()

C:\Users\v_kly\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [49]:
fetures_not_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 279085 entries, 2 to 354368
Data columns (total 10 columns):
 #   Column             Non-Null Count   Dtype   
---  ------             --------------   -----   
 0   VehicleType        279085 non-null  category
 1   RegistrationYear   279085 non-null  int64   
 2   Gearbox            279085 non-null  category
 3   Power              279085 non-null  int64   
 4   Model              279085 non-null  category
 5   Kilometer          279085 non-null  int64   
 6   RegistrationMonth  279085 non-null  int64   
 7   FuelType           279085 non-null  category
 8   Brand              279085 non-null  category
 9   NotRepaired        279085 non-null  category
dtypes: category(6), int64(4)
memory usage: 12.5 MB


Разбиваем на тренировочную, валидационную и тестовую выборку.

In [50]:
features_train_not_encoded, features_valid_test_not_encoded, target_train_not_encoded, target_valid_test_not_encoded = train_test_split(
    fetures_not_encoded, target, test_size=0.4, random_state=12345)

features_valid_not_encoded, features_test_not_encoded, target_valid_not_encoded, target_test_not_encoded = train_test_split(
    features_valid_test_not_encoded, target_valid_test_not_encoded, test_size=0.5, random_state=12345)

In [51]:
print(features_train_not_encoded.shape, target_train_not_encoded.shape)
print(features_valid_not_encoded.shape, target_valid_not_encoded.shape)
print(features_test_not_encoded.shape, target_test_not_encoded.shape)

(167451, 10) (167451,)
(55817, 10) (55817,)
(55817, 10) (55817,)


##### Модель LightGBM с категориальными признаками

Создаем и обучаем модель.

In [52]:
model_lgbm_category = LGBMRegressor(n_estimators=500)

In [53]:
%%time
model_lgbm_category.fit(features_train_not_encoded, target_train_not_encoded)

Wall time: 1.27 s


LGBMRegressor(n_estimators=500)

In [54]:
%%time
pred_lgbm = model_lgbm_category.predict(features_valid_not_encoded)

Wall time: 236 ms


In [55]:
rmse_lgbm = mean_squared_error(target_valid_not_encoded, pred_lgbm)**0.5

In [56]:
print('RMSE модели', rmse_lgbm)

RMSE модели 1505.2787289283956


Время обучения модели LGBM с использованием категориальных признаков составило ``5 минут 59 секунд``. Значение метрики RMSE равно ``1533.54``

##### Модель CatBoost с категориальными признаками

In [57]:
cat_not_encoded = CatBoostRegressor(learning_rate=0.1 , depth=6, cat_features = fetures_not_encoded[category])

In [58]:
%%time
cat_not_encoded.fit(features_train_not_encoded, target_train_not_encoded, verbose=100)

0:	learn: 4319.5803990	total: 171ms	remaining: 2m 51s
100:	learn: 1733.4757174	total: 8.77s	remaining: 1m 18s
200:	learn: 1647.4809643	total: 17.4s	remaining: 1m 9s
300:	learn: 1603.2308254	total: 26s	remaining: 1m
400:	learn: 1576.3892215	total: 34.7s	remaining: 51.8s
500:	learn: 1555.5896015	total: 43.3s	remaining: 43.2s
600:	learn: 1537.6034831	total: 52.3s	remaining: 34.8s
700:	learn: 1522.2861654	total: 1m 1s	remaining: 26.4s
800:	learn: 1509.9631082	total: 1m 10s	remaining: 17.6s
900:	learn: 1500.1671020	total: 1m 19s	remaining: 8.75s
999:	learn: 1488.8734911	total: 1m 28s	remaining: 0us
Wall time: 1min 29s


In [59]:
%%time
pred_cat = cat_not_encoded.predict(features_valid_not_encoded)

Wall time: 103 ms


In [60]:
rmse_cat = mean_squared_error(target_valid_not_encoded, pred_cat)**0.5

In [61]:
print('RMSE модели', rmse_cat)

RMSE модели 1554.9278241009592


Время обучения модели CatBoost на с использованием категориальных признаков составило ``3 минут 42 секунд``. Значение метрики RMSE равно ``1578.67``

**Вывод:**

In [62]:
index_lgbm = {
    'Использование категориальных признаков',
    'Использование кодирования данных'    
}
col_lgbm = {'Время обучения': [359, 26], 
       'RMSE': [1533.54, 1659.79]}
research_models_lgbm = pd.DataFrame(col_lgbm, index=index_lgbm)
print('Модель LGBM')
display(research_models_lgbm)

Модель LGBM


,Время обучения,RMSE
Использование кодирования данных,359,1533.54
Использование категориальных признаков,26,1659.79


In [63]:
index_cat = {
    'Использование категориальных признаков',
    'Использование кодирования данных'    
}
col_cat = {'Время обучения': [222, 116], 
       'RMSE': [1578.67, 1670.05]}
research_models_cat = pd.DataFrame(col_cat, index=index_cat)
print('Модель CatBoost')
display(research_models_cat)

Модель CatBoost


,Время обучения,RMSE
Использование кодирования данных,222,1578.67
Использование категориальных признаков,116,1670.05


Как видно из двух таблиц выше, модели при использовании категориальных признаков обучаются дольше, но метрика RMSE показывает лучший результат. В финальных тестированиях будет использовать эти модели.

## Анализ моделей

Возьмем лучшие итерации по каждому типу моделей с определенными параметрами и запустим их на тестовой выборке. 

А затем сравним результаты работы и время обучения

### Модель случайного леса

In [64]:
%%time
best_model_rfr.fit(features_train, target_train)

Wall time: 3min 40s


RandomForestRegressor(max_depth=17, n_estimators=80)

In [65]:
%%time
predictions_rfr = best_model_rfr.predict(features_test)

Wall time: 895 ms


In [66]:
rmse_rfr = mean_squared_error(target_test, predictions_rfr)**0.5
print('RMSE модели случайного леса:', rmse_rfr)

RMSE модели случайного леса: 1626.7809167905903


### Модель LightGBM

In [67]:
%%time
pred_lgbm_test = model_lgbm_category.predict(features_test_not_encoded)

Wall time: 222 ms


In [68]:
rmse_lgbm_test = mean_squared_error(target_test, pred_lgbm_test)**0.5
print('RMSE модели LightGBM:', rmse_lgbm_test)

RMSE модели LightGBM: 1526.2720167012562


### Модель CatBoost

In [69]:
%%time
pred_cat = cat_not_encoded.predict(features_test_not_encoded)

Wall time: 112 ms


In [70]:
rmse_cat_test = mean_squared_error(target_test, pred_cat)**0.5
print('RMSE модели CatBoost:', rmse_cat_test)

RMSE модели CatBoost: 1576.289886476883


## Вывод

In [71]:
index = [
    ['RandomForestRegression', 125, 1.57, 1656.32],
    ['LightGBM', 359, 3, 1542.57],
    ['CatBoost', 222, 0.743, 1585.78]
]
col_cat = ['Модель', 'Время обучения (сек)', 'Время предсказания (сек)', 'RMSE']
research_models_cat = pd.DataFrame(index, columns=col_cat)
print('Модель CatBoost')
display(research_models_cat)

results = pd.DataFrame()

Модель CatBoost


,Модель,Время обучения (сек),Время предсказания (сек),RMSE
0,RandomForestRegression,125,1.570,1656.32
1,LightGBM,359,3.000,1542.57
2,CatBoost,222,0.743,1585.78


Лучшая скорость обучения у модели RandomForestRegression, при этом метрика качества занимает третье место. Данная модель подойдет для систем с постоянным переобучением и не очень высокими требованиями к величине ошибки.

Лучшая метрика качества у модели LightGBM, при этом у неё самое большое время на обучение и предсказание, данная модель подходит для систем требовательных к качеству прогнозирования, но где не требуется постоянного переобучения и прогнозирования.

Модель CatBoost имеет средний показатель по метрике качества, но при этом показатель скорости прогнозирования лучший в два и четыре раза по отношению к моделям RandomForestRegression и LightGBM соответственно, что позволяет использовать эту модель в системах с высокой нагрузкой постоянного прогнозирования, без переобучения в реальном времени.
